In [ ]:
#Coefficient of Determination

This is a useful coefficient for isolating attributes by detrending data, the r2_score yields a euclidean measurement for the degree of separation of the predicted outcome relative to the true outcome using a hermitian basis.  

In [ ]:
def r2_score(ytrue, ypred):
  return 1 - sum((ytrue-ypred)**2)/sum((mean(ytrue)-ytrue)**2)

def RSS(y_residue):
    return 1/2 * np.sum(y_residue**2)

Least Squares for (debiasing)

In [ ]:
#TRAINING DATA
n1,m1=Xtrain.shape
X_ones1=np.ones((n1,1))
X1=np.column_stack((X_ones1,Xtrain))

q1,r1 = qr(X1)
w1 = solve(r1, q1.T.dot(ytrain))
yhat1=X1.dot(w1)

#TESTING DATA
n2,m2=Xtest.shape
X_ones2=np.ones((n2,1))
X2=np.column_stack((X_ones2,Xtest))
yhat2=X2.dot(w1) #predicted y

#R^2 test
print('R2 for Least Square for training is ', r2_score(ytrain,yhat1))
print('R2 for Least Square is ', r2_score(ytest,yhat2))
print('RSS for Least Square is', RSS(ytest-yhat2))

Ridge Regression Using Coefficient of Determination (bias)

In [ ]:
l_span   = linspace(0,2,30)
w1_store = zeros((30, 9))
error_store = zeros(30)
r2_store    = zeros(30)
for index in range(len(l_span)):
    l         = l_span[index]
    sq_Lambda = diag(ones(9)*sqrt(l)) 
    X1_tilde  = np.vstack([X1, sq_Lambda])
    ytrain_tilde   = append(ytrain, zeros(9))
    q, r       = linalg.qr(X1_tilde)
    w1_tilde    = linalg.solve(r, q.T.dot(ytrain_tilde))
    w1_store[index,:] = w1_tilde
    w1_hat=np.array([w1_tilde])
    yhat1_test = X2.dot(w1_hat.T) 
    error_store[index] = RSS(ytest-yhat1_test)
    r2_store[index]= r2_score(ytest, yhat1_test)

fig, ax = plt.subplots()
ax.scatter(l_span, error_store)
ylabel('RSS')
xlabel('$\lambda$')
title('RSS Value for $\lambda$ Values')
grid()
print('min of RSS in lambda is', min(error_store))  
print('optimal lambda is ', l_span[argmin(error_store)])
print('r2 in optimal lambda is', r2_store[argmin(error_store)]) 

LASSO

In [ ]:
def lasso(X, y, l1, tol=1e-6):
"""The Lasso Regression model
      Pathwise coordinate descent with co-variance updates is applied.

      X - NumPy matrix, size (N, d), of standardized numerical predictors, note the first column is ones.
      y - NumPy array, length N, of numerical response.
      l1 - L1 penalty tuning parameter (positive scalar)
      tol - Coordinate Descent convergence tolerance (exited if change < tol)

  """
    m, n      = np.shape(X)
    q, r      = linalg.qr(X);
    w_s       = linalg.solve(r, q.T.dot(y))
    iter   = 0
    while True:
        w_star = w_s.copy()
        for j in range(n):
       # norm_X_j = LA.norm(X[:, j])
       # selector = [i for i in range(X.shape[1]) if i != j]
       # a = X[:, j].dot(y[:, np.newaxis] - X[:, selector].dot(w_s[:, np.newaxis][selector, :]))
       # res = np.sign(a) * max(abs(a) - l1, 0)   
       # w_s[j] = res/(norm_X_j**2)

            a_j     = LA.norm(X[:, j])**2
            index   = arange(n)
            index_d = delete(index, j)
            c_j     = np.dot(X[:,j].T, y-np.dot(X[:,index_d],w_s[index_d]))
            update  = c_j/a_j
            w_s[j]  = np.sign(update) * max(abs(update) - l1/a_j, 0)   
        
          iter += 1
            if np.all(abs(w_s - w_star) < tol):
        #print('Number of iteration is ', iter)
        break

          return w_s

In [ ]:
#plot lasso path
l_span   = linspace(0,50,100)
w4_store = zeros((100, 9))

for index in range(len(l_span)):
    l         = l_span[index]
    w         = lasso(X1,ytrain, l, tol=1e-6)
    w4_store[index,:]=w.T

fig, ax = plt.subplots()
for index in range(8):
    ax.plot(l_span, w4_store[:,index+1],'-',label= f'{names[index]}')
leg = ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left');

In [ ]:
w=lasso(X1,ytrain,5,tol=1e-06)
print(w)

In [ ]:
from scipy.io import loadmat

!wget https://github.com/yexf308/MAT592/blob/main/homework/HW1/regression_outlier.mat?raw=true -O regression_outlier.mat
data = loadmat('regression_outlier.mat')
X, Y=data['x_train'], data['y_train']
fig, ax = plt.subplots()
plt.plot(X,Y,'o',color='black')

In [ ]:
from scipy.optimize import minimize

def fun_1(x):
  return sum(abs(Y-x[0]-X*x[1]))

m=minimize(fun_1, [0,0])
a = m.x

def fun_2(x):
  return sum((Y-x[0]-X*x[1])**2)

n=minimize(fun_2, [0,0])
b = n.x

fig, ax = plt.subplots()
plt.plot(X,Y,'o',color='black')
x = np.linspace(0,12,100)
plot(x, b[1]*x+b[0], '-b', label='LS regression')
plot(x, a[1]*x+a[0], '-r', label='LAD regression')
leg = ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left');
plt.show()